# Analysieren der Daten in csv Datei um Ähnlichkeitsmaße zu entwickeln und zu implementieren

Inhalt des Notebooks 

1. [Analyse der Daten](#Analyse-der-Daten)
2. [Entwicklung eines Ähnlichkeitmaßes für die versch. Ausprägungen](#Entwicklung-eines-Ähnlichkeitmaßes-für-die-versch.-Ausprägungen)
3. [Klassifikationslogik](#Klassifikationslogik)
4. [Programmablauf Aufbau Datenbasis](#Programmablauf-Aufbau-Datenbasis)
5. [Weitere Ausführungen](#Weitere-Ausführungen)

## Analyse der Daten

In [12]:
import pandas as pd

# CSV-Datei einlesen
df = pd.read_csv('Wohnungskartei_1.csv', sep=';')

In [13]:
# Für jede Spalte die verschiedenen Ausprägungen ausgeben
for column in df.columns:
    print(f"Spalte '{column}' hat {len(df[column].unique())} Asprägungen: ")
    print(f"{df[column].unique()} \n\n")


Spalte 'Zimmerzahl' hat 10 Asprägungen: 
['Zwei Zimmer' '4 Zimmer' 'Ein Zimmer' '6 Zimmer' '2-3 Zimmer' '5 Zimmer'
 '3-4 Zimmer' '4-5 Zimmer' '5-6 Zimmer' '3 Zimmer'] 


Spalte 'Stockwerk' hat 11 Asprägungen: 
['1.Stock' '3.Stock' 'EG' '2.Stock' '5.Stock' '4.Stock' '8.Stock'
 '9.Stock' '6.Stock' '7.Stock' '10.Stock'] 


Spalte 'Heizung' hat 6 Asprägungen: 
['Gas-Zentral' 'Gas-Etage' 'Öl' 'Pellets' 'Erdwärme' 'Lüftungsheizung'] 


Spalte 'Hausmeister' hat 2 Asprägungen: 
['nein' 'ja'] 


Spalte 'Kindergarten' hat 3 Asprägungen: 
['nah' 'fern' 'erreichbar'] 


Spalte 'Schule' hat 3 Asprägungen: 
['erreichbar' 'nah' 'fern'] 


Spalte 'S-Bahn' hat 4 Asprägungen: 
['nah' 'erreichbar' 'nein' 'mit Bus'] 


Spalte 'Garage' hat 2 Asprägungen: 
['nein' 'ja'] 


Spalte 'Miete' hat 13 Asprägungen: 
['251-300' '551-600' '751-800' '401-450' '201-250' '651-700' '501-550'
 '601-650' '801-850' '701-750' '451-500' '901-950' '851-900'] 


Spalte 'Nebenkosten' hat 7 Asprägungen: 
['50-100' '201-250' '151-

In [14]:
# Dicts zum Mappen der Ausprägungen auf numerische Werte

# Spalte 'Zimmerzahl'
zimmerzahl_dict = {'Zwei Zimmer': 2, '4 Zimmer': 4, 'Ein Zimmer': 1, '6 Zimmer': 6, '2-3 Zimmer': 2.5, '5 Zimmer': 5,
 '3-4 Zimmer': 3.5, '4-5 Zimmer': 4.5, '5-6 Zimmer': 5.5, '3 Zimmer': 3}

stockwerk_dict = {'1.Stock': 1, '3.Stock': 3, 'EG': 0, '2.Stock': 2, '5.Stock': 5, '4.Stock': 4, '8.Stock': 8, '9.Stock': 9,
 '6.Stock': 6, '7.Stock': 7, '10.Stock': 10}

# heizung_dict_Nominal = {'Gas-Zentral': 1, 'Gas-Etage': 2, 'Öl': 3, 'Pellets': 4, 'Erdwärme': 5, 'Lüftungsheizung': 6} # TODO: ! Macht keine Sinn hier eine Ordnung zu definieren

# hausmeister_dict = {'nein': 0, 'ja': 1}

kindergarten_dict = {'nah': 1, 'erreichbar': 2, 'fern': 3}
# Distanzen wobei 'nah' die geringste Distanz, 'erreichbar' eine mittlere und 'fern' die größte Distanz bezeichnet.

schule_dict = {'nah': 1, 'erreichbar': 2, 'fern': 3}

sbahn_dict = {'nah': 1, 'erreichbar': 2, 'mit Bus': 3, 'nein': 4}

# garage_dict = {'nein': 0, 'ja': 1}

miete_dict = {'251-300': 275, '551-600': 575, '751-800': 775, '401-450': 425, '201-250': 225, '651-700': 675, '501-550': 525,
 '601-650': 625, '801-850': 825, '701-750': 725, '451-500': 475, '901-950': 925, '851-900': 875}

nebenkosten_dict = {'50-100': 75, '201-250': 225, '151-200': 175, 'unter 50': 25, '251-300': 275, '101-150': 125, 'über 300': 325}


alter_dict = {'16-20 Jahre': 18, '4-7 Jahre': 5.5, '7-10 Jahre': 8.5, '11-15 Jahre': 13, '31-50 Jahre': 40.5, 'ueber 100': 100,
 'Neubau': 0, '51-75 Jahre': 63, '1-3 Jahre': 2, '76-100 Jahre': 88, '21-30 Jahre': 25.5}

# aufzug_dict = {'ja': 1, 'nein': 0}

#TODO: ! Macht keine Sinn hier eine Ordnung zu definieren
# lage_dict_Nominal = {'Hauptstrasse': 1, 'Wohngebiet': 2, 'Abgelegen': 3, 'Spielstrasse': 4, 'Nebenstrasse': 5}

entfernung_dict = {'< 1 km': 0.5, '5-10 km': 7.5, '21-30 km': 25.5, '< 3 km': 1.5, '3-5 km': 4, '> 30 km': 35.5, '11-20 km': 15.5}

kaution_dict = {'keine': 0, 'ueber 3000': 3500, '1000-1500': 1250}

# kueche_dict_Nominal = {'Küche (alt)': 1, 'Küche (neu)': 2, 'keine': 0}


# bad_dict_Nominal = {'Dusche': 1, 'Badewanne': 2, 'Dusche + Wanne': 3, 'Dusche auf Flur': 4, 'Waschbecken': 5}

# balkon_dict = {'nein': 0, 'ja': 1}
# 
# terrasse_dict = {'nein': 0, 'ja': 1}
# 
# kehrwoche_dict = {'nein': 0, 'ja': 1}
# 
moebilisiert_dict = {'nein': 0, 'teilmoebliert': 1, 'ja': 2}

quadratmeter_dict = {'31-40': 35.5, '91-100': 95.5, '81-90': 85.5, '20-30': 25.5, 'über 120': 125, '51-60': 55.5, '101-110': 105.5,
 '71-80': 75.5, '111-120': 115.5, '61-70': 65.5, '41-50': 45.5}

# kabelanschluss_dict = {'nein': 0, 'ja': 1}

deckenhoehe_dict = {'250': 250, '260': 260, '280':280, '300':300}

# abstellraum_dict = {'ja': 1, 'nein': 0}

# waschraum_dict = {'nein': 0, 'ja': 1}

# klasse_dict = {'Studierende': 0, 'DINK':1, 'Kleinfamilie':2, 'Single':3, 'Expatriate':4}


## Entwicklung eines Ähnlichkeitmaßes für die versch. Ausprägungen

In [15]:
def similiarity_simple(x, y) -> float:
    diff = abs(x - y)
    # Berechnung der Ähnlichkeit als inverse Funktion der Differenz
    similarity = 1 / (1 + diff)
    
    return similarity

def similiarity_yes_no(x, y) -> bool:
    if x == y:
        return True
    return False

def similiarity_relative_to_interval(dict_values: dict, x, y) -> float:
    diff = abs(x - y)
    max_distance = max(dict_values.values()) - min(dict_values.values())
    relative_distance = diff / max_distance
    return 1 - relative_distance
    

## Klassifikationslogik

1. Berechnung der Ähnlichkeiten: Berechnen der Ähnlichkeit zu jedem Fall in der Fallbasis.
2. Auswahl ähnlichster Fälle: Auswahl des oder der ähnlichsten Fälle:
    - Auswahl des einzigen ähnlichsten Falls.
    - Auswahl der Top-N ähnlichsten Fälle.
    - Auswahl aller Fälle, die eine bestimmte Ähnlichkeitsschwelle überschreiten.

In [16]:
count = df['Klasse'].value_counts()
print(count)

Klasse
Kleinfamilie    128
Studierende      83
DINK             40
Expatriate       10
Single            4
Name: count, dtype: int64


Da die Klassen im Datensatz nicht gleichmäßig aufgeteilt sind macht es keinen Sinn mehrere ähnliche Fälle auszuwählen.
Beispiel: Wähle Top 5 Ähnlichste Fälle. Zwar ist Single der ähnlichste Fall aber Kleinfamilie ist sehr häufig und könnte somit die anderen 4 Plätze besetzen. Es macht deshalb aber keinen Sinn von Kleinfamilie auszugehen.

--> Wähle nur den Ähnlichsten Fall aus!

## Programmablauf Aufbau Datenbasis:

Für jeden Datenpunkt:   
1. Ist ein Datenpunkt in der Fallbasis? Wenn nicht hinzufügen und abbrechen
2. Ähnlichsten Fall finden -> Lösungsvorschlag für Mieterkategorie
3. Wenn richtig, abbrechen
4. Wenn falsch, -> Hinzufügen zu Fallbasis


Zusatz: Probieren ob verschiedene Gewichte und Ähnlichkeitsmaße bessere Ergebnisse liefern.


## Berechnung der Ähnlichkeiten zwischen den Fällen

- Für jedes Attribut wird die Ähnlichkeit berechnet
- Die Ähnlichkeiten werden aufsummiert
- Je nach Attribut mit unterschiedlichen Ähnlichkeitsmaßen und ggf. Berücksichtigung der Alternativenanzahl / Wichtigkeit (Informiertes Ähnlichkeitsmaß)
- Normieren der Ähnlichkeiten auf [0,1] durch Division der maximalen Ähnlichkeit ( Davor berechnet)

In [17]:
def get_absolute_similiarity_to_case(case1:dict, case2:dict) -> float:
    # Berechnung der Ähnlichkeiten für jeden Fall in der Fallbasis
    similarity = 0
    for key, value in case1.items():
        if case2[key]:
            if key == 'Zimmerzahl':
                # Informierte Ähnlichkeit: Berücksichtigung der Alternativenanzahl
                similarity += similiarity_relative_to_interval(zimmerzahl_dict, zimmerzahl_dict[value], zimmerzahl_dict[case2[key]]) * len(zimmerzahl_dict.keys())
            elif key == 'Stockwerk':
                similarity += similiarity_relative_to_interval(stockwerk_dict, stockwerk_dict[value], stockwerk_dict[case2[key]])
            elif key == 'Heizung':
                similiarity_yes_no(value, case2[key])
            elif key == 'Hausmeister':
                similarity += similiarity_yes_no(value, case2[key])
            elif key == 'Kindergarten':
                similarity += similiarity_relative_to_interval(kindergarten_dict, kindergarten_dict[value], kindergarten_dict[case2[key]])
            elif key == 'Schule':
                similarity += similiarity_relative_to_interval(schule_dict, schule_dict[value], schule_dict[case2[key]])
            elif key == 'S-Bahn':
                similarity += similiarity_relative_to_interval(sbahn_dict, sbahn_dict[value], sbahn_dict[case2[key]])
            elif key == 'Garage':
                similarity += similiarity_yes_no(value, case2[key])
            elif key == 'Miete':
                similarity += similiarity_simple(miete_dict[value], miete_dict[case2[key]])
            elif key == 'Nebenkosten':
                similarity += similiarity_simple(nebenkosten_dict[value], nebenkosten_dict[case2[key]])
            elif key == 'Alter':
                similarity += similiarity_simple(alter_dict[value], alter_dict[case2[key]])
            elif key == 'Entfernung':
                similarity += similiarity_simple(entfernung_dict[value], entfernung_dict[case2[key]])
            elif key == 'Kaution':
                similarity += similiarity_relative_to_interval(kaution_dict ,kaution_dict[value], kaution_dict[case2[key]])
            elif key == 'Deckenhöhe':
                # print(value)
                similarity += similiarity_relative_to_interval(deckenhoehe_dict, (deckenhoehe_dict[str(value)]), (deckenhoehe_dict[(str(case2[key]))]))
            elif key == 'Moebliert':
                similarity += similiarity_relative_to_interval(moebilisiert_dict, moebilisiert_dict[value], moebilisiert_dict[case2[key]])
            elif key == 'Quadratmeter':
                similarity += similiarity_simple(quadratmeter_dict[value], quadratmeter_dict[case2[key]])
            elif key == 'Kueche':
                similarity += similiarity_yes_no(value, case2[key])
            elif key == 'Waschraum':
                similarity += similiarity_yes_no(value, case2[key])
            elif key == 'Balkon':
                similarity += similiarity_yes_no(value, case2[key])
            elif key == 'Abstellraum':
                similarity += similiarity_yes_no(value, case2[key])
            elif key == 'Aufzug':
                similarity += similiarity_yes_no(value, case2[key])
            elif key == 'Lage':
                similarity += similiarity_yes_no(value, case2[key])
            elif key == 'Kabelanschluss':
                similarity += similiarity_yes_no(value, case2[key])
            elif key == 'Terrasse':
                similarity += similiarity_yes_no(value, case2[key])
            elif key == 'Kehrwoche':
                similarity += similiarity_yes_no(value, case2[key])
            elif key == 'Bad':
                similarity += similiarity_yes_no(value, case2[key])
            elif key == 'Klasse':
                # print('Info: Klasse angegeben und ignoriert.')
                pass
            else:
                print(f"Key '{key}' not found!")
    return similarity

In [18]:
def get_similiarity_to_every_case(wohnung:dict, df:pd.DataFrame) -> pd.DataFrame:

    max_similarity = get_absolute_similiarity_to_case(wohnung, wohnung)
    # Berechnung der Ähnlichkeiten für jeden Fall in der Fallbasis
    highest_similiarity = 0
    index_of_highest_similiarity = None
    # keine Ähnlichkeit für die Ausprägung berechnen wenn wohnung nicht diese Ausprägung hat
    for i in range(len(df)):
        abs_similarity = get_absolute_similiarity_to_case(wohnung, dict(df.iloc[i]))
        y = abs_similarity / max_similarity
        # print(f"{abs_similarity}, {max_similarity}, {y}")
        if y > highest_similiarity:
            highest_similiarity = y
            index_of_highest_similiarity = i
    return index_of_highest_similiarity
                    

# Fallbasis bauen:

In [20]:


# langsam füllen von df in fallbasis
# df durchmischen um zufällige Reihenfolge zu erhalten
# Damit nicht immer die gleichen Fälle hinzugefügt werden --> Zufällige Reihenfolge führt zu unterschiedlichen Fällen ---> Unterschiedliche Fallbasis --> Untershciedliche Ergebnisse
# Dafür aber eben ein "echteres" Bild vom Fallbasisaufbau
df = df.sample(frac=1).reset_index(drop=True)
fallbasis = pd.DataFrame(columns=df.columns)

def fallbasis_bauen(df:pd.DataFrame, fallbasis) -> pd.DataFrame:
    # über df iterieren 
    for i in range(len(df)):
        # wenn fallbasis leer ist, füge ersten Fall hinzu
        if len(fallbasis) == 0:
            fallbasis = pd.concat([fallbasis, df.iloc[i:i+1]], ignore_index=True)
            print(f"Fall {i} hinzugefügt.")
        else:
            # wenn fallbasis nicht leer ist, berechne Ähnlichkeit zu jedem Fall in fallbasis
            index_of_most_similiar_case = get_similiarity_to_every_case(dict(df.iloc[i]), fallbasis)
            # finde ähnlichsten Fall
            most_similiar_case = dict(fallbasis.iloc[index_of_most_similiar_case])
            
            # wenn ähnlichster Fall nicht die gleiche Klasse hat, füge Fall hinzu
            if most_similiar_case['Klasse'] != df.iloc[i]['Klasse']:
                fallbasis = pd.concat([fallbasis, df.iloc[i:i+1]], ignore_index=True)
                print(f"Fall {i} hinzugefügt.")
            else:
                print(f"Fall {i} nicht hinzugefügt.")
    return fallbasis

fallbasis = fallbasis_bauen(df, fallbasis)
print(len(fallbasis))

Fall 0 hinzugefügt.
Fall 1 hinzugefügt.
Fall 2 hinzugefügt.
Fall 3 hinzugefügt.
Fall 4 nicht hinzugefügt.
Fall 5 nicht hinzugefügt.
Fall 6 nicht hinzugefügt.
Fall 7 nicht hinzugefügt.
Fall 8 hinzugefügt.
Fall 9 nicht hinzugefügt.
Fall 10 nicht hinzugefügt.
Fall 11 nicht hinzugefügt.
Fall 12 hinzugefügt.
Fall 13 nicht hinzugefügt.
Fall 14 nicht hinzugefügt.
Fall 15 nicht hinzugefügt.
Fall 16 nicht hinzugefügt.
Fall 17 nicht hinzugefügt.
Fall 18 nicht hinzugefügt.
Fall 19 nicht hinzugefügt.
Fall 20 hinzugefügt.
Fall 21 hinzugefügt.
Fall 22 hinzugefügt.
Fall 23 nicht hinzugefügt.
Fall 24 nicht hinzugefügt.
Fall 25 hinzugefügt.
Fall 26 nicht hinzugefügt.
Fall 27 nicht hinzugefügt.
Fall 28 nicht hinzugefügt.
Fall 29 hinzugefügt.
Fall 30 hinzugefügt.
Fall 31 nicht hinzugefügt.
Fall 32 nicht hinzugefügt.
Fall 33 nicht hinzugefügt.
Fall 34 nicht hinzugefügt.
Fall 35 nicht hinzugefügt.
Fall 36 nicht hinzugefügt.
Fall 37 nicht hinzugefügt.
Fall 38 hinzugefügt.
Fall 39 nicht hinzugefügt.
Fall 40 

In [21]:
# nochmal durchlaufen lassen zum Überprüfen --> Neue Fälle die hinzugefügt wurden können Fälle am Anfang der Fallbasis jetzt ähnlicher sein
fallbasis = fallbasis_bauen(df, fallbasis)

Fall 0 nicht hinzugefügt.
Fall 1 nicht hinzugefügt.
Fall 2 nicht hinzugefügt.
Fall 3 nicht hinzugefügt.
Fall 4 nicht hinzugefügt.
Fall 5 nicht hinzugefügt.
Fall 6 nicht hinzugefügt.
Fall 7 nicht hinzugefügt.
Fall 8 nicht hinzugefügt.
Fall 9 nicht hinzugefügt.
Fall 10 nicht hinzugefügt.
Fall 11 nicht hinzugefügt.
Fall 12 nicht hinzugefügt.
Fall 13 nicht hinzugefügt.
Fall 14 nicht hinzugefügt.
Fall 15 nicht hinzugefügt.
Fall 16 hinzugefügt.
Fall 17 nicht hinzugefügt.
Fall 18 nicht hinzugefügt.
Fall 19 nicht hinzugefügt.
Fall 20 nicht hinzugefügt.
Fall 21 nicht hinzugefügt.
Fall 22 nicht hinzugefügt.
Fall 23 nicht hinzugefügt.
Fall 24 nicht hinzugefügt.
Fall 25 nicht hinzugefügt.
Fall 26 nicht hinzugefügt.
Fall 27 nicht hinzugefügt.
Fall 28 nicht hinzugefügt.
Fall 29 nicht hinzugefügt.
Fall 30 nicht hinzugefügt.
Fall 31 hinzugefügt.
Fall 32 nicht hinzugefügt.
Fall 33 nicht hinzugefügt.
Fall 34 nicht hinzugefügt.
Fall 35 nicht hinzugefügt.
Fall 36 nicht hinzugefügt.
Fall 37 nicht hinzugefü

In [22]:
print(len(fallbasis))

54


## Weitere Ausführungen

### Mapping der Ausprägungen auf numerische Werte
Das Mapping der Ausprägung ist notwendig und sinnvoll bei vielen Attributen, da die Ähnlichkeit bei diesen Attributen zwischen zwei Ausprägungen des gleichen Attributs nicht nur 0 oder 1 ist. Beispiel: Die Ähnlichkeit zwischen 1 und 2 ist höher als die Ähnlichkeit zwischen 1 und 3.   
Dies kann abgebildet werden, indem die Ausprägungen auf numerische Werte gemappt werden. Die Ähnlichkeit zwischen zwei Ausprägungen ist dann die Differenz der numerischen Werte geteilt durch die maximale Differenz (bzw. ggf. relativ zur maximalen Differenz). Die Ähnlichkeit zwischen einer Ausprägung und sich selbst ist damit natürlich auch 1.

### Auswahl der Ähnlichkeitsmaße
Auswahl der Ähnlichkeitsmaße in diesem Projekt basiert auf der Art der Attribute, die Relevanz der Attribute und dem Probieren verschiedener Maße. 

### Gesamtähnlichkeit 
Die Gesamtähnlichkeit zwischen zwei Fällen ist die Summe der Ähnlichkeiten ihrer Attribute (Bei uns wird auch noch normalisiert, wenn auch nicht notwendig für die Funktionsfähigkeit). Nachdem die Ähnlichkeiten berechnet wurden, wird der ähnlichste Fall in der Fallbasis gefunden. Die Mieterkategorie dieses Falles wird als Vorhersage für die Mieterkategorie des neuen Falles verwendet.

### Informiertes Ähnlichkeitsmaß
Nach Probieren und Testen stellte sich heraus das in den meisten Fällen informierte Ähnlichkeitsmaße nicht die Resultate verbessern bzw. die Fallbasis sogar vergrößern. Deshalb wurde darauf verzichtet.

### Fallbasis
Die Fallbasis wird langsam aufgebaut. Für jeden neuen Fall wird die Ähnlichkeit zu jedem Fall in der Fallbasis berechnet. Der ähnlichste Fall wird gefunden und dessen Mieterkategorie wird als Vorhersage für die Mieterkategorie des neuen Falles verwendet. Wenn die Vorhersage falsch ist, wird der neue Fall zur Fallbasis hinzugefügt. Dadurch wird die Fallbasis langsam aufgebaut und die Vorhersagen werden immer genauer, wobei die Fallbasis trotzdem möglichst klein gehalten wird.    
( retrieve reuse revise retain )    
Es ist sinnvoll alle Fälle mehrmals durchzugehen. Neue Fälle die hinzugefügt wurden, können Fälle die am Anfang der Fallbasis nicht hinzugefügt wurden, jetzt ähnlicher sein und damit kann es nötig sein, sie beim zweiten Durchlauf hinzuzufügen.    
Beispiel: Der erste Fall wird immer hinzugefügt. Mit hoher Wahrscheinlichkeit ist es Klasse "Kleinfamilie" (fast 50% der Datenpunkte sind Klasse "Kleinfamilie"). Wenn der zweite Fall auch Klasse "Kleinfamilie" ist, wird er nie hinzugefügt weil der ähnlichste Fall ist immer schon Klasse "Kleinfamilie".